In [1]:
import sys
import os
import pandas as pd
import re

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [4]:
# nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
# ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host, filter_dev_data
import warnings


config_data = read_yaml_for_host("sst_config.yml")



python initialized for apply_loocv_and_save
cpus available; cpus to use:
28 25
28
yaml file does not contain host-specific settings for this host. Using default settings.


In [5]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"
dropbox_datapath = config_data['dropbox_datapath']


In [6]:
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20230102T164214.csv")
train_subjs = filter_dev_data(test_train_df_raw, nonbids_data_path,dropbox_datapath)


In [7]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

from dev_wtp_io_utils import import_sst_cond_w1_subjs_to_pkl

In [8]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

### With supplementary data

In [9]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1
#individual_differences['wave'] = individual_differences['wave'].astype(object) # for compatibility with the wave column in the dataset

In [10]:
from outlier_detection import *

In [11]:
"', '".join(individual_differences.columns)

"subject', 'cancer_promoting_minus_preventing_FCI', 'cancer_promoting_minus_preventing_FFQ', 'cancer_promoting_minus_preventing_craved_FCI', 'cancer_promoting_minus_preventing_craved_FFQ', 'cancer_promoting_minus_preventing_liked_FCI', 'cancer_promoting_minus_preventing_liked_FFQ', 'cancer_promoting_FCI', 'cancer_promoting_FFQ', 'cancer_preventing_FCI', 'cancer_preventing_FFQ', 'BSCS', 'cSES', 'EDM', 'BIS_11', 'PCS', 'RS', 'TRSQ', 'bf_1', 'weight_0', 'height_0', 'birthsex', 'age365', 'bmi_0', 'bmi_1', 'bmi', 'bf_1_controlled', 'bf_1_bsexnormedzs', 'ACES_sum', 'ACES_household_dysfunction', 'ACES_neglectful_parenting', 'ACES_abuse', 'ACES_divorced_separated', 'BFI_agreeableness', 'BFI_conscientiousness', 'BFI_extraversion', 'BFI_neuroticism', 'BFI_openness', 'DEMO_mcarthur_social_standing', 'IMI_effort_importance', 'IMI_interest_enjoyment', 'IMI_perceived_choice', 'IMI_perceived_competence', 'IMI_value_usefulness', 'IPAQ_moderateminutes', 'IPAQ_sittinghours', 'IPAQ_vigorousminutes', 'IPA

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:4],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond'
)

In [25]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_spatially_concatenated',
    concatenate_condition_labels=True
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV005     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV005.nii
  subject  wave
0  DEV005     1
...imported data.
temp mask deleted
DEV006
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

               

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV019.nii
  subject  wave
0  DEV019     1
...imported data.
temp mask deleted
DEV020
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV020     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/g

KeyboardInterrupt: 

In [32]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV005     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.ni

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV017     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV017     1  
...imported data.
temp mask deleted
DEV018
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV018     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV018     1  
.....verified that all expect

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV027     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV027     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV027     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV027     1  
...imported data.
temp mask deleted
DEV028
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV040     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV040     1  
...imported data.
temp mask deleted
DEV041
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV041     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV041     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV051     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV051     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV051     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV051     1  
...imported data.
temp mask deleted
DEV052
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV060     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV060     1  
...imported data.
temp mask deleted
DEV062
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV062     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV062     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV076     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV076     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV076     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV076     1  
...imported data.
temp mask deleted
DEV077
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV089     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV089     1  
...imported data.
temp mask deleted
DEV097
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV097     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV097     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV159     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV159     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV159     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV159     1  
...imported data.
temp mask deleted
DEV164
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV206     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV206     1  
...imported data.
temp mask deleted
DEV215
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV215     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV215     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

## concatenating conditions

In [ ]:
from nilearn import plotting
from nilearn.masking import compute_brain_mask
import nltools as nlt
import nilearn as nil

In [ ]:
from nltools import Brain_Data

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_spatially_concatenated',
    concatenate_condition_labels=True
)

In [ ]:
out_data = pickle.load(open(
    '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_2subs_correct_cond_w_supplementary_mask_fix.pkl','rb'
))

In [ ]:
out_data.shape

In [ ]:
plotting.plot_glass_brain(out_data[0,].to_nifti())

## post-error conditions

In [27]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq

train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [28]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_go_fcs_w_supplementary'
)

In [29]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_correct_stop_cg_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   

   wave  
0     1  
...imported data.
temp mask deleted
DEV006
   condition_index                condition_label  \
0              

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV018   

   wave  
0     1  
...imported data.
temp mask deleted
DEV019
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV019   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description         

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV046   

   wave  
0     1  
...imported data.
temp mask deleted
DEV047
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV047   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV057   

   wave  
0     1  
...imported data.
temp mask deleted
DEV058
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV058   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV073   

   wave  
0     1  
...imported data.
temp mask deleted
DEV074
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV074   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV089   

   wave  
0     1  
...imported data.
temp mask deleted
DEV097
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV097   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV164   

   wave  
0     1  
...imported data.
temp mask deleted
DEV169
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV169   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV217   

   wave  
0     1  
...imported data.
temp mask deleted
DEV218
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV218   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

In [30]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingFailedStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_failed_stop_cg_failed_stop_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
1     1  
...imported data.
temp mask deleted
DEV006
   condition_index               condition_label  \
1                1  

   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV017   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV017   

   wave  
1     1  
...imported data.
temp mask deleted
DEV018
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta

   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV027   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV027   

   wave  
1     1  
...imported data.
temp mask deleted
DEV028
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta

   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV042   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV042   

   wave  
1     1  
...imported data.
temp mask deleted
DEV043
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV055   

   wave  
1     1  
...imported data.
temp mask deleted
DEV056
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV056   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV069   

   wave  
1     1  
...imported data.
temp mask deleted
DEV071
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV071   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV086   

   wave  
1     1  
...imported data.
temp mask deleted
DEV087
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV087   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV158   

   wave  
1     1  
...imported data.
temp mask deleted
DEV159
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV159   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV215   

   wave  
1     1  
...imported data.
temp mask deleted
DEV216
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV216   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

In [31]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_stop_cg_spatially_concatenated',
    concatenate_condition_labels=True
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV005.nii
  subject  wave
0  DEV005     1
...imported data.
temp mask deleted
DEV006
   condition_index                con

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV018.nii
  subject  wave
0  DEV018     1
...imported data.
temp mask deleted
DEV019
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV019   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV019   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

  subject  wave
0  DEV028     1
...imported data.
temp mask deleted
DEV029
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV029   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV029   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV029.nii
  subject  wave
0  DEV029     1
...imported data.
temp mask deleted
DEV030
   condition_index                condition_label  \
0        

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV043.nii
  subject  wave
0  DEV043     1
...imported data.
temp mask deleted
DEV046
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV046   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV046   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV056.nii
  subject  wave
0  DEV056     1
...imported data.
temp mask deleted
DEV057
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV057   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV057   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV071.nii
  subject  wave
0  DEV071     1
...imported data.
temp mask deleted
DEV073
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV073   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV073   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV087   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV087   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV087.nii
  subject  wave
0  DEV087     1
...imported data.
temp mask deleted
DEV089
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoF

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV158.nii
  subject  wave
0  DEV158     1
...imported data.
temp mask deleted
DEV159
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV159   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV159   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV215.nii
  subject  wave
0  DEV215     1
...imported data.
temp mask deleted
DEV216
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV216   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV216   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

## no smoothing

In [12]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV020
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV020     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_

In [15]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:6],'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV006
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV006     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV006     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV006     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_

In [13]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:30],'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV020
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV020     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV100     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV100     1  
...imported data.
temp mask deleted
DEV102
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV102     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV102     1  
.....verified that all exp

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV119     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV119     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV119     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV119     1  
...imported data.
temp mask deleted
DEV135
   condition_index conditi

In [16]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs,'conditions_nos',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':"beta"
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions_nos/
DEV006
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV006     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV006     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV006     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV021     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV021     1  
...imported data.
temp mask deleted
DEV022
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV022     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV022     1  
.....verified that all exp

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV033     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV033     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV033     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV033     1  
...imported data.
temp mask deleted
DEV034
   condition_index conditi

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV043     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV043     1  
...imported data.
temp mask deleted
DEV044
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV044     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV044     1  
.....verified that all exp

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV053     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV053     1  
...imported data.
temp mask deleted
DEV054
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV054     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV054     1  
.....verified that all exp

...

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV254     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV254     1  
...imported data.
temp mask deleted
DEV255
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV255     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV255     1  
.....verified that all exp

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV267     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV267     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV267     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV267     1  
...imported data.
temp mask deleted
DEV268
   condition_index conditi

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV278     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV278     1  
...imported data.
temp mask deleted
DEV281
not enough conditions for subject DEV281. Skipping this subject.
DEV284
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV284     1  
1  spm_spm:beta (0002) - Sn(1) Cor

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV296     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV296     1  
.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV296     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV296     1  
...imported data.
temp mask deleted
DEV297
   condition_index conditi

.....verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV309     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV309     1  
...imported data.
temp mask deleted
DEV310
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV310     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV310     1  
.....verified that all exp